In [96]:
# from pyglet.window import key
import numpy as np
from agar.Env import AgarEnv
import time
import torch
import torch.nn as nn
import torch.optim as optim
render = False
num_agents = 1


class Args:
    def __init__(self):

        self.num_controlled_agent = num_agents
        self.num_processes = 64
        self.action_repeat = 1
        self.total_step = 1e8
        self.r_alpha = 0.1
        self.r_beta = 0.1
        self.seed = 42
        self.gamma = 0.99
        self.eval = True

def calculate_reward(observations):
    total_rewards = []
    for agent in observations:
        if observations[agent]["metadata"]["is_killed"]:
            observations[agent] = 0
            continue
        reward = 0
        for food in observations[agent]["food"]:
            reward += 1 / food["relative_position_x^2 + relative_position_y^2"]
        total_rewards.append(reward)
    return total_rewards


In [97]:
class Memory:
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.logprobs = []
        self.values = []
        self.next_value = None

    def clear(self):
        del self.states[:]
        del self.actions[:]
        del self.rewards[:]
        del self.logprobs[:]
        del self.values[:]
        del self.masks[:]
        self.next_value = None

In [102]:
class TransformerModel(nn.Module):
    def __init__(self,input_dim, dim_hidden, output_dim):
        super().__init__()
        self.fc1 = torch.nn.Sequential(
            torch.nn.Linear(input_dim, dim_hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(dim_hidden, dim_hidden)
        )
        self.transformer = nn.TransformerEncoderLayer(d_model=dim_hidden, nhead=8)
        self.fc2 = torch.nn.Sequential(
            torch.nn.Linear(dim_hidden, output_dim)
        )
    def forward(self, x):
        out = self.fc1(x)
        out = out.permute(1, 0, 2)
        out = self.transformer(out)
        out = out.permute(1, 0, 2)
        out = self.fc2(out)
        return out

In [103]:
agent_transformer = TransformerModel(578, 512, 3)
critic_transformer = TransformerModel(578, 512, 1)